https://mp.weixin.qq.com/s/0qOGevwTltu2SJ-QanW0VA

# 工具包导入&数据读取
## 工具包导入

In [1]:
import numpy as np
import pandas as pd
import os
from datetime import date, timedelta
from time import time
from tqdm import tqdm_notebook, tqdm
from sklearn.metrics import f1_score
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import StratifiedKFold, KFold, RepeatedKFold
from sklearn.svm import NuSVR, SVR
from sklearn.linear_model import LinearRegression
from sklearn.kernel_ridge import KernelRidge
from sklearn.metrics import mean_absolute_error

import lightgbm as lgb
import xgboost as xgb
from catboost import CatBoostRegressor

import gc

from scipy.signal import hilbert
from scipy.signal import hann
from scipy.signal import convolve
from scipy import stats

from dateutil.parser import parse

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

import warnings
warnings.filterwarnings("ignore")

from itertools import combinations, product

import ast

## 数据读取

### air_reserve
- air_store_id - the restaurant's id in the air system
- visit_datetime - the time of the reservation
- reserve_datetime - the time the reservation was made
- reserve_visitors - the number of visitors for that reservation

### hpg_reserve
- hpg_store_id - the restaurant's id in the hpg system
- visit_datetime - the time of the reservation
- reserve_datetime - the time the reservation was made
- reserve_visitors - the number of visitors for that reservation

### air_store_info
- air_store_id
- air_genre_name
- air_area_name
- latitude
- longitude

### hpg_store_info
- hpg_store_id
- hpg_genre_name
- hpg_area_name
- latitude
- longitude

### date_info
- calendar_date
- day_of_week
- holiday_flg - is the day a holiday in Japan

### store_id_relation
This file allows you to join select restaurants that have both the air and hpg system.
- hpg_store_id
- air_store_id

### air_visit_data
- air_store_id
- visit_date - the date
- visitors - the number of visitors to the restaurant on the date

### sample_submission
This file shows a submission in the correct format, including the days for which you must forecast.

- id - the id is formed by concatenating the air_store_id and visit_date with an underscore
- visitors - the number of visitors forecasted for the store and date combination

In [2]:
path = '../input/'

air_visit_data      =  pd.read_csv(path + 'air_visit_data.csv') 

hpg_store_info      =  pd.read_csv(path + 'hpg_store_info.csv')
air_store_info      =  pd.read_csv(path + 'air_store_info.csv')

hpg_reserve         =  pd.read_csv(path + 'hpg_reserve.csv')
air_reserve         =  pd.read_csv(path + 'air_reserve.csv') 

date_info           =  pd.read_csv(path + 'date_info.csv')  

store_id_relation   =  pd.read_csv(path + 'store_id_relation.csv')     

sample_submission   =  pd.read_csv(path + 'sample_submission.csv')

**air_visit_data.csv**

- air_store_id
- visit_date - the date
- visitors - the number of visitors to the restaurant on the date

In [3]:
air_visit_data.head()

,air_store_id,visit_date,visitors
0,air_ba937bf13d40fb24,2016-01-13,25
1,air_ba937bf13d40fb24,2016-01-14,32
2,air_ba937bf13d40fb24,2016-01-15,29
3,air_ba937bf13d40fb24,2016-01-16,22
4,air_ba937bf13d40fb24,2016-01-18,6


**hpg_store_info.csv**

- hpg_store_id
- hpg_genre_name
- hpg_area_name
- latitude
- longitude

Note: latitude and longitude are the latitude and longitude of the area to which the store belongs

In [4]:
hpg_store_info.head()

,hpg_store_id,hpg_genre_name,hpg_area_name,latitude,longitude
0,hpg_6622b62385aec8bf,Japanese style,Tōkyō-to Setagaya-ku Taishidō,35.643675,139.668221
1,hpg_e9e068dd49c5fa00,Japanese style,Tōkyō-to Setagaya-ku Taishidō,35.643675,139.668221
2,hpg_2976f7acb4b3a3bc,Japanese style,Tōkyō-to Setagaya-ku Taishidō,35.643675,139.668221
3,hpg_e51a522e098f024c,Japanese style,Tōkyō-to Setagaya-ku Taishidō,35.643675,139.668221
4,hpg_e3d0e1519894f275,Japanese style,Tōkyō-to Setagaya-ku Taishidō,35.643675,139.668221


**air_store_info.csv**

- air_store_id
- air_genre_name
- air_area_name
- latitude
- longitude

Note: latitude and longitude are the latitude and longitude of the area to which the store belongs

In [5]:
air_store_info.head()

,air_store_id,air_genre_name,air_area_name,latitude,longitude
0,air_0f0cdeee6c9bf3d7,Italian/French,Hyōgo-ken Kōbe-shi Kumoidōri,34.695124,135.197852
1,air_7cc17a324ae5c7dc,Italian/French,Hyōgo-ken Kōbe-shi Kumoidōri,34.695124,135.197852
2,air_fee8dcf4d619598e,Italian/French,Hyōgo-ken Kōbe-shi Kumoidōri,34.695124,135.197852
3,air_a17f0778617c76e2,Italian/French,Hyōgo-ken Kōbe-shi Kumoidōri,34.695124,135.197852
4,air_83db5aff8f50478e,Italian/French,Tōkyō-to Minato-ku Shibakōen,35.658068,139.751599


**hpg_reserve.csv**

This file contains reservations made in the hpg system.

- hpg_store_id - the restaurant's id in the hpg system
- visit_datetime - the time of the reservation
- reserve_datetime - the time the reservation was made
- reserve_visitors - the number of visitors for that reservation

In [6]:
hpg_reserve.head()

,hpg_store_id,visit_datetime,reserve_datetime,reserve_visitors
0,hpg_c63f6f42e088e50f,2016-01-01 11:00:00,2016-01-01 09:00:00,1
1,hpg_dac72789163a3f47,2016-01-01 13:00:00,2016-01-01 06:00:00,3
2,hpg_c8e24dcf51ca1eb5,2016-01-01 16:00:00,2016-01-01 14:00:00,2
3,hpg_24bb207e5fd49d4a,2016-01-01 17:00:00,2016-01-01 11:00:00,5
4,hpg_25291c542ebb3bc2,2016-01-01 17:00:00,2016-01-01 03:00:00,13


**air_reserve.csv**
- air_store_id - the restaurant's id in the air system
- visit_datetime - the time of the reservation
- reserve_datetime - the time the reservation was made
- reserve_visitors - the number of visitors for that reservation

In [7]:
air_reserve.head()

,air_store_id,visit_datetime,reserve_datetime,reserve_visitors
0,air_877f79706adbfb06,2016-01-01 19:00:00,2016-01-01 16:00:00,1
1,air_db4b38ebe7a7ceff,2016-01-01 19:00:00,2016-01-01 19:00:00,3
2,air_db4b38ebe7a7ceff,2016-01-01 19:00:00,2016-01-01 19:00:00,6
3,air_877f79706adbfb06,2016-01-01 20:00:00,2016-01-01 16:00:00,2
4,air_db80363d35f10926,2016-01-01 20:00:00,2016-01-01 01:00:00,5


**date_info.csv**
- calendar_date
- day_of_week
- holiday_flg - is the day a holiday in Japan

In [8]:
date_info.head()

,calendar_date,day_of_week,holiday_flg
0,2016-01-01,Friday,1
1,2016-01-02,Saturday,1
2,2016-01-03,Sunday,1
3,2016-01-04,Monday,0
4,2016-01-05,Tuesday,0


**store_id_relation.csv**

This file allows you to join select restaurants that have both the air and hpg system.

- hpg_store_id
- air_store_id

In [9]:
store_id_relation.head()

,air_store_id,hpg_store_id
0,air_63b13c56b7201bd9,hpg_4bc649e72e2a239a
1,air_a24bf50c3e90d583,hpg_c34b496d0305a809
2,air_c7f78b4f3cba33ff,hpg_cd8ae0d9bbd58ff9
3,air_947eb2cae4f3e8f2,hpg_de24ea49dc25d6b8
4,air_965b2e0cf4119003,hpg_653238a84804d8e7


**sample_submission.csv**

This file shows a submission in the correct format, including the days for which you must forecast.

- id - the id is formed by concatenating the air_store_id and visit_date with an underscore
- visitors - the number of visitors forecasted for the store and date combination

In [10]:
sample_submission.head()

,id,visitors
0,air_00a91d42b08b08d9_2017-04-23,0
1,air_00a91d42b08b08d9_2017-04-24,0
2,air_00a91d42b08b08d9_2017-04-25,0
3,air_00a91d42b08b08d9_2017-04-26,0
4,air_00a91d42b08b08d9_2017-04-27,0


## 数据合并

In [11]:
sample_submission['air_store_id'] = sample_submission['id'].apply(lambda x: x[:-11]).values
sample_submission['visit_date'] = sample_submission['id'].apply(lambda x: x[-10:]).values

del sample_submission['id']
gc.collect()

60

In [12]:
df_meta = pd.concat([air_visit_data, sample_submission],
                    axis=0,
                    ignore_index=True)

# 合并date_info
df_meta = df_meta.merge(date_info,
                        left_on=['visit_date'],
                        right_on=['calendar_date'],
                        how='left')
# 合并air_store_info
df_meta = df_meta.merge(air_store_info,
                        on=['air_store_id'],
                        how='left')

del df_meta['calendar_date'], df_meta['day_of_week']
gc.collect()

0

In [13]:
df_meta['visit_date'] = pd.to_datetime(df_meta['visit_date'])
# Monday=0, Sunday=6
df_meta['weekday'] = df_meta['visit_date'].dt.weekday

### 类别层次特征

In [14]:
df_meta['air_store_id_weekday'] = df_meta['air_store_id'].astype(str) + '_' + df_meta['weekday'].astype(str)

df_meta['air_store_id_holiday'] = df_meta['air_store_id'].astype(str) + '_' + df_meta['holiday_flg'].astype(str)
df_meta['air_store_id_weekday_holiday'] = df_meta['air_store_id_holiday'].astype(str) + '_' + df_meta['weekday'].astype(str)

df_meta['air_genre_name_weekday'] = df_meta['air_area_name'].astype(str) + df_meta['weekday'].astype(str)
df_meta['air_genre_name_weekday_holiday'] = df_meta['air_genre_name_weekday'].astype(str) + df_meta['holiday_flg'].astype(str)

In [15]:
df_meta['visitors'] = df_meta['visitors'].apply(np.log1p).values

df_meta.head()

,air_store_id,visit_date,visitors,holiday_flg,air_genre_name,air_area_name,latitude,longitude,weekday,air_store_id_weekday,air_store_id_holiday,air_store_id_weekday_holiday,air_genre_name_weekday,air_genre_name_weekday_holiday
0,air_ba937bf13d40fb24,2016-01-13,3.258097,0,Dining bar,Tōkyō-to Minato-ku Shibakōen,35.658068,139.751599,2,air_ba937bf13d40fb24_2,air_ba937bf13d40fb24_0,air_ba937bf13d40fb24_0_2,Tōkyō-to Minato-ku Shibakōen2,Tōkyō-to Minato-ku Shibakōen20
1,air_ba937bf13d40fb24,2016-01-14,3.496508,0,Dining bar,Tōkyō-to Minato-ku Shibakōen,35.658068,139.751599,3,air_ba937bf13d40fb24_3,air_ba937bf13d40fb24_0,air_ba937bf13d40fb24_0_3,Tōkyō-to Minato-ku Shibakōen3,Tōkyō-to Minato-ku Shibakōen30
2,air_ba937bf13d40fb24,2016-01-15,3.401197,0,Dining bar,Tōkyō-to Minato-ku Shibakōen,35.658068,139.751599,4,air_ba937bf13d40fb24_4,air_ba937bf13d40fb24_0,air_ba937bf13d40fb24_0_4,Tōkyō-to Minato-ku Shibakōen4,Tōkyō-to Minato-ku Shibakōen40
3,air_ba937bf13d40fb24,2016-01-16,3.135494,0,Dining bar,Tōkyō-to Minato-ku Shibakōen,35.658068,139.751599,5,air_ba937bf13d40fb24_5,air_ba937bf13d40fb24_0,air_ba937bf13d40fb24_0_5,Tōkyō-to Minato-ku Shibakōen5,Tōkyō-to Minato-ku Shibakōen50
4,air_ba937bf13d40fb24,2016-01-18,1.945910,0,Dining bar,Tōkyō-to Minato-ku Shibakōen,35.658068,139.751599,0,air_ba937bf13d40fb24_0,air_ba937bf13d40fb24_0,air_ba937bf13d40fb24_0_0,Tōkyō-to Minato-ku Shibakōen0,Tōkyō-to Minato-ku Shibakōen00


## 预定表，包括air_reserve和hpg_reserve

In [16]:
# air系统
air_reserve['visit_date'] = air_reserve['visit_datetime'].apply(lambda x: x.split(' ')[0]).values
air_reserve['air_store_id_visit_date'] = air_reserve['air_store_id'] + '_' + air_reserve['visit_datetime'].astype(str)

air_reserve['visit_datetime'] = pd.to_datetime(air_reserve['visit_datetime'])
air_reserve['reserve_datetime'] = pd.to_datetime(air_reserve['reserve_datetime'])

# hpg系统
hpg_reserve['visit_date'] = hpg_reserve['visit_datetime'].apply(lambda x: x.split(' ')[0]).values


# hpg系统关联得到air_store_id
hpg_to_air_storeid = store_id_relation.set_index('hpg_store_id')
hpg_reserve['air_store_id'] = hpg_reserve['hpg_store_id'].map(hpg_to_air_storeid['air_store_id']).values
hpg_air_reserve = hpg_reserve.loc[hpg_reserve.air_store_id.notnull()]

hpg_air_reserve['air_store_id_visit_date'] = hpg_air_reserve['air_store_id'] + '_' + hpg_air_reserve['visit_date'].astype(str)
hpg_air_reserve['visit_datetime'] = pd.to_datetime(hpg_air_reserve['visit_datetime'])
hpg_air_reserve['reserve_datetime'] = pd.to_datetime(hpg_air_reserve['reserve_datetime'])

# 特征工程
## 一阶特征
### store_id的特征

1. store_id的基础信息，包括类型，地址，经纬度
2. store_id的营业天数，开业到现在的时间，最后一天到现在的时间

In [17]:
def get_storeid_fea(df_visit_data, df_store_info, df_store_id_relation):
    df_storeid = df_store_info.copy()
    
    # 对air_genre_name和air_area_name做LabelEncoder
    df_storeid['air_genre_name'] = LabelEncoder().fit_transform(df_storeid['air_genre_name'].values)
    df_storeid['air_area_name'] = LabelEncoder().fit_transform(df_storeid['air_area_name'].values)
    
    # 每个air_store_id日期的最小值和最大值
    first_day_dict = df_visit_data.groupby(['air_store_id'])['visit_date'].min().to_dict()
    last_day_dict = df_visit_data.groupby(['air_store_id'])['visit_date'].max().to_dict()
    
    # 每个air_store_id的营业天数
    run_days_dict = df_visit_data.groupby(['air_store_id'])['visit_date'].nunique().to_dict()
    
    # 每个air_store_id (2017-04-22 - first_day)相差的天数
    df_storeid['first_day_to_now'] = df_storeid['air_store_id'].map(first_day_dict).values
    df_storeid['first_day_to_now'] = df_storeid['first_day_to_now'].apply(lambda x: (parse('2017-04-22') - parse(x)).days)
    
    # 每个air_store_id (2017-04-22 - last_day)相差的天数
    df_storeid['last_day_to_now'] = df_storeid['air_store_id'].map(last_day_dict).values
    df_storeid['last_day_to_now'] = df_storeid['last_day_to_now'].apply(lambda x: (parse('2017-04-22') - parse(x)).days)
    
    # 每个air_store_id的营业天数
    df_storeid['run_days'] = df_storeid['air_store_id'].map(run_days_dict).values
    
    # air_store_id是否在store_id_relation里面
    df_storeid['two_systems'] = df_storeid['air_store_id'].isin(df_store_id_relation['air_store_id']).values
    df_storeid['two_systems'] = df_storeid['two_systems'].astype(int)
    
    return df_storeid

In [18]:
df_storeid = get_storeid_fea(air_visit_data, air_store_info, store_id_relation)

### date的特征
1. 星期几
2. 是否节假日
3. 距离月末，月初的时间
4. 距离最近的节假日的时间

In [19]:
def get_last_next_holiday(dates, holiday_flags):
    last_holiday = []
    next_holiday = []
    len_ = len(dates)
    for i, (d, f) in enumerate(zip(dates, holiday_flags)):
        if i == 0:
            last_holiday.append(dates[i])
        else:
            # last holiday
            last_flag = 0
            for j in range(i - 1, 0, -1):
                if holiday_flags[j] == 1:
                    last_holiday.append(dates[j])
                    last_flag = 1
                    break
            if last_flag == 0:
                last_holiday.append(dates[0])
                
        # next holiday
        next_flag = 0
        for j in range(i + 1, len_):
            if holiday_flags[j] == 1:
                next_holiday.append(dates[j])
                next_flag = 1
                break
        if next_flag == 0:
            next_holiday.append(dates[-1])
    return last_holiday, next_holiday

In [20]:
def get_days_to_month_end(x):
    """
    计算每个日期距离月底的天数
    """
    month = int(x.split('-')[1])
    day = int(x.split('-')[2])
    if month in [1, 3, 5, 7, 8, 10, 12]:
        return 31 - day
    elif month in [2]:
        return 28 - day
    else:
        return 30 - day

In [21]:
def get_time_fea(df_date_info):
    """
    date特征
    1. 每个日期距月底、月初的天数
    2. 每个日期是周几，是否是周末
    3. 每个日期距离上一个和下一个节假日的天数
    4. 每个日期的前几天、后几天是否是节假日
    """
    df_date = df_date_info.copy()
    
    # 每个日期距离月底的天数
    df_date['day_to_month_end'] = df_date['calendar_date'].apply(lambda x: get_days_to_month_end(x)).values
    # 每个日期距离月初的天数
    # df_date['day_to_month_start'] = df_date['calendar_date'].apply(lambda x: int(x.split('-')[2])).values
    df_date['day_to_month_start'] = df_date['calendar_date'].apply(lambda x: int(x.split('-')[0])).values
    
    # 每个日期是周几
    df_date['day_of_week'] = df_date['calendar_date'].apply(lambda x: parse(x).weekday()).values
    # 每个日期是否是周末
    df_date['is_weekend'] = df_date['day_of_week'].apply(lambda x: x > 4).values
    
    # 每个日期的上一个和下一个节假日日期
    last_holiday, next_holiday = get_last_next_holiday(df_date['calendar_date'].values, df_date['holiday_flg'].values)
    df_date['last_holiday'] = last_holiday
    df_date['next_holiday'] = next_holiday

    # 每个日期距离上一个节假日的天数
    df_date['day_to_last_holiday'] = (df_date['calendar_date'].apply(lambda x: parse(x))
                                      - df_date['last_holiday'].apply(lambda x: parse(x)))
    df_date['day_to_last_holiday'] = df_date['day_to_last_holiday'].apply(lambda x: x.days).values
    
    # 每个日期距离下一个节假日的天数
    df_date['day_to_next_holiday'] = (df_date['next_holiday'].apply(lambda x: parse(x))
                                      - df_date['calendar_date'].apply(lambda x: parse(x)))
    df_date['day_to_next_holiday'] = df_date['day_to_next_holiday'].apply(lambda x: x.days).values
    
    df_date.rename(columns={'calendar_date': 'visit_date'}, inplace=True)
    
    df_date['visit_date'] = pd.to_datetime(df_date['visit_date'])
    
    # 每个日期的前几天，后几天是否是节假日
    for i in [3, 2, 1, -1]:
        date_info_temp = df_date.copy()
        # visit_date = visit_date + i
        date_info_temp['visit_date'] = date_info_temp['visit_date'].apply(lambda x: x + timedelta(i))
        date_info_temp.rename(columns={'holiday_flg': 'ahead_holiday_{}'.format(i)},
                              inplace=True)
        
        df_date = df_date.merge(date_info_temp[['visit_date', 'ahead_holiday_{}'.format(i)]],
                                on=['visit_date'],
                                how='left')
    del df_date['last_holiday'], df_date['next_holiday']
    gc.collect()
    return df_date

In [22]:
df_date = get_time_fea(date_info)

## 二阶特征：store_id & day

因为day这个东西只有一个,而且未来和之前的是不一致的,那么做特征的时候就需要找day相关的属性,同样的store_id的属性也可以构建特征

### store_id+day(related特征/holiday/weekday)

In [23]:
def get_st_date(en_date, days):
    """
    对每个日期减几天
    """
    # st_date <= date < end_date
    st_date = en_date - timedelta(days)
    return st_date

def get_label_st_date(st_date, days):
    """
    对每个日期加几天
    """
    # st_date <= date < end_date
    end_date = st_date + timedelta(days)
    return st_date, end_date

def get_df_label(df, date, days=39, is_te=False):
    st_date, en_date = get_label_st_date(date, days)
    print('label date, ', st_date, en_date)
    ind1 = df['visit_date'] < en_date
    ind2 = df['visit_date'] >= st_date
    if is_te:
        return df.loc[(ind2), ['air_store_id', 'visit_date', 'air_store_id_holiday',
                                'air_genre_name', 'air_genre_name_weekday', 'air_genre_name_weekday_holiday']].copy()
    return df.loc[(ind1 & ind2), ['air_store_id', 'visit_date', 'air_store_id_holiday',
                                   'air_genre_name', 'air_genre_name_weekday', 'air_genre_name_weekday_holiday', 'visitors']].copy()

### 特征工程1：visitors相关特征
#### air_store_id+visitors特征（recent days）

最近$N(N=14,28,56,1000)$天air_store_id的统计特征

In [24]:
def get_store_id_visitors_sts_features(df, en_date, key='air_store_id', date_col='visit_date', days_list=[1000, 56, 28, 14,]):
    """
    时间滑窗
    计算每个窗口内，每个店铺的visitors的统计量
    """
    t0 = time()
    df_features       = pd.DataFrame()
    df_features[key]  = df[key].unique()

    df_tmp            = df.copy()
    #######################  Recent days sts features #######################
    for days in days_list:
        # 时间窗口
        st_date           = get_st_date(en_date, days)  # 对日期减几天 
        df_tmp            = df_tmp.loc[((df[date_col] >= st_date) &  (df[date_col] < en_date))].copy() 
        df_features_tmp   = df_tmp.groupby([key], as_index=False)['visitors'].agg({
            '{}_visitors_min_{}'.format(key,days): 'min',
            '{}_visitors_mean_{}'.format(key,days): 'mean',
            '{}_visitors_sum_{}'.format(key,days): 'sum',
            '{}_visitors_median_{}'.format(key,days): 'median',
            '{}_visitors_max_{}'.format(key,days): 'max',
            '{}_visitors_count_{}'.format(key,days): 'count',
            '{}_visitors_std_{}'.format(key,days): 'std',
            '{}_visitors_quantile_{}'.format(key,days): 'quantile',
            '{}_visitors_skew_{}'.format(key,days): 'skew'
        }) 
        df_features  = df_features.merge(df_features_tmp, on =key, how = 'left')
    print('For store id sts features, we spend {} seconds.'.format(time() - t0)) 

    return df_features

#### air_store_id + visitors diff特征(recent days)
最近N(N=28,56,1000)天的趋势统计特征

In [25]:
def get_store_id_visitors_diff_sts_features(df, en_date, key='air_store_id', date_col='visit_date', days_list=[1000, 56, 28]):
    """
    visitors_diff的统计量
    1. 时间窗口
    2. 对每个日期减1，在用日期去关联，相当于得到昨天的visitors
    3. visitors_diff = 当天的vistors - 昨天的visitors
    4. visitors_diff的统计量
    5. visitors_diff的绝对值
    6. visitors_diff的绝对值的平均值
    """
    t0 = time()
    df_features = pd.DataFrame()
    df_features[key] = df[key].unique()
    
    for days in days_list:
        df_tmp = df.copy()
        st_date = get_st_date(en_date, days)  # 对日期减几天
        # 时间窗口
        df_tmp = df_tmp.loc[((df[date_col] >= st_date) & (df[date_col] < en_date))].copy()
        df_old_tmp = df_tmp.copy()
        
        # 每个air_store_id昨天的visitors
        df_old_tmp[date_col] = df_tmp[date_col].apply(lambda x: x + timedelta(1)).values  # 用昨天的日期去关联
        df_old_tmp.rename(columns={'visitors': 'visitors_yesterday'},
                          inplace=True)
        df_tmp = df_tmp.merge(df_old_tmp[[key, date_col, 'visitors_yesterday']],
                              on=[key, date_col],
                              how='left')
        
        # 当天的visitors - 昨天的visitors
        df_tmp['visitors_diff'] = df_tmp['visitors'].values - df_tmp['visitors_yesterday'].values
        
        df_tmp = df_tmp.loc[df_tmp['visitors_diff'].notnull()]
        
        df_features_tmp = df_tmp.groupby([key], as_index=False)['visitors_diff'].agg({
            '{}_visitors_diff_min_{}'.format(key, days): 'min',
            '{}_visitors_diff_mean_{}'.format(key, days): 'mean',
            '{}_visitors_diff_median_{}'.format(key, days): 'median',
            '{}_visitors_diff_sum_{}'.format(key, days): 'sum',
            '{}_visitors_diff_max_{}'.format(key, days): 'max',
            '{}_visitors_diff_count_{}'.format(key, days): 'count',
            '{}_visitors_diff_std_{}'.format(key, days): 'std',
            '{}_visitors_diff_quantile_{}'.format(key, days): 'quantile',
            '{}_visitors_diff_skew_{}'.format(key, days): 'skew',
        })
        
        df_tmp['visitors_diff_abs'] = df_tmp['visitors_diff'].map(abs).values
        
        # 绝对值的平均值
        df_features_tmp2 = df_tmp.groupby([key], as_index=False)['visitors_diff_abs'].agg({
            '{}_visitors_diff_abs_mean_{}'.format(key, days): 'mean'
        })
        
        df_features = df_features.merge(df_features_tmp, on=key, how='left')
        df_features = df_features.merge(df_features_tmp2, on=key, how='left')
    print('For store id diff sts features, we spend {} seconds.'.format(time() - t0))
    
    return df_features

#### air_store_id + weekday + visitors特征(周期特征)
过去每段时间的工作日的的统计特征

In [26]:
def get_weekday_visitors_sts_features(df, key, end_date, date_col='visit_date', val_name='visitor'):
    """
    时间滑窗，groupby对象不同，这里用的是key=air_store_id_weekday
    1. 时间滑窗，前4、8、12、100周
    2. 时间窗内的visitor的统计量
    3. 最大值-最小值
    """
    df_features = pd.DataFrame()
    df_features[key] = df[key].unique()
    
    # 日期小于end_date
    ind1 = df[date_col] < end_date
    
    # 时间滑窗，日期前4周，8周，12周，100周
    for i in [4, 8, 12, 100]:
        t_st_date = end_date - timedelta(7 * i)
        ind2 = df[date_col] >= t_st_date
        df_grp = df.loc[(ind1 & ind2)].groupby(key)
        dic_mean = df_grp[val_name].mean().to_dict()
        dic_std = df_grp[val_name].std().to_dict()
        dic_max = df_grp[val_name].max().to_dict()
        dic_min = df_grp[val_name].min().to_dict()
        dic_count = df_grp[val_name].count().to_dict()
        dic_sum = df_grp[val_name].sum().to_dict()
        dic_quantile = df_grp[val_name].quantile().to_dict()
        
        df_features['{}_weekdays_mean_{}_{}'.format(key, i + 1, val_name)] = df_features[key].map(dic_mean).values
        df_features['{}_weekdays_std_{}_{}'.format(key, i + 1, val_name)] = df_features[key].map(dic_std).values
        df_features['{}_weekdays_max_{}_{}'.format(key, i + 1, val_name)] = df_features[key].map(dic_max).values
        df_features['{}_weekdays_min_{}_{}'.format(key, i + 1, val_name)] = df_features[key].map(dic_min).values
        df_features['{}_weekdays_count_{}_{}'.format(key, i + 1, val_name)] = df_features[key].map(dic_count).values
        df_features['{}_weekdays_sum_{}_{}'.format(key, i + 1, val_name)] = df_features[key].map(dic_sum).values
        df_features['{}_weekdays_quantile_{}_{}'.format(key, i + 1, val_name)] = df_features[key].map(dic_quantile).values
        
        # 最大值 - 最小值
        df_features['{}_weekdays_gap_{}_{}'.format(key, i + 1, val_name)] = (df_features['{}_weekdays_max_{}_{}'.format(key, i + 1, val_name)] -
                                                                             df_features['{}_weekdays_min_{}_{}'.format(key, i + 1, val_name)])
        
        return df_features

#### air_store_id + weekday + visitors diff特征(周期特征)
过去每段时间的工作日的差值统计特征

In [27]:
def get_weekday_visitors_diff_sts_features(df, key, end_date, date_col='visit_date', val_name='visitors'): 
    """
    时间滑窗，groupby对象不同，这里用的是key=air_store_id_weekday
    1. 时间滑窗，前4周、8周、12周、100周
    2. 前1周那天的visitors_diff
    3. visitors_diff的统计量
    """
    df_features = pd.DataFrame() 
    df_features[key] = df[key].unique()

    ind1 = df[date_col] < end_date  
    # 时间滑窗，前4周，8周，12周，100周
    for i in [4, 8, 12, 100]:
        t_st_date = end_date - timedelta(7 * i) 
        ind2 = df[date_col] >= t_st_date 

        df_tmp = df.loc[ind1 & ind2].copy() 

        df_old_tmp = df_tmp.copy()
        # 每个air_store_id 前1周的那天的visitors
        df_old_tmp[date_col] = df_tmp[date_col].apply(lambda x: x + timedelta(7)).values
        df_old_tmp.rename(columns={'visitors': 'visitors_lastweek'},
                          inplace=True)
        df_tmp = df_tmp.merge(df_old_tmp[[key, date_col, 'visitors_lastweek']],
                              on=[key, date_col],
                              how='left')
        # 当天visitors - 前1周的那天的visitors
        df_tmp['visitors_diff'] = df_tmp['visitors'].values - df_tmp['visitors_lastweek'].values
        df_tmp = df_tmp.loc[df_tmp['visitors_diff'].notnull()]

        df_tmp['visitors_diff_abs'] = df_tmp['visitors_diff'].map(abs).values

        df_grp              = df_tmp.groupby(key) 
        dic_mean            = df_grp['visitors_diff'].mean().to_dict()
        dic_std             = df_grp['visitors_diff'].std().to_dict()
        dic_max             = df_grp['visitors_diff'].max().to_dict()
        dic_min             = df_grp['visitors_diff'].min().to_dict()
        dic_count           = df_grp['visitors_diff'].count().to_dict()
        dic_sum             = df_grp['visitors_diff'].sum().to_dict()   
        dic_quantile        = df_grp['visitors_diff'].quantile().to_dict()   
        dic_abs_mean        = df_grp['visitors_diff_abs'].mean().to_dict()   
        
        df_features['{}_weekday_quantile_{}_{}'.format(key, i + 1, 'visitorsdiff')]  = df_features[key].map(dic_quantile).values 
        df_features['{}_weekday_mean_{}_{}'.format(key, i + 1, 'visitorsdiff')]      = df_features[key].map(dic_mean).values 
        df_features['{}_weekday_std_{}_{}'.format(key, i + 1, 'visitorsdiff')]       = df_features[key].map(dic_std).values 
        df_features['{}_weekday_max_{}_{}'.format(key, i + 1, 'visitorsdiff')]       = df_features[key].map(dic_max).values 
        df_features['{}_weekday_min_{}_{}'.format(key, i + 1, 'visitorsdiff')]       = df_features[key].map(dic_min).values 
        df_features['{}_weekday_count_{}_{}'.format(key, i + 1, 'visitorsdiff')]     = df_features[key].map(dic_count).values 
        df_features['{}_weekday_sum_{}_{}'.format(key, i + 1, 'visitorsdiff')]       = df_features[key].map(dic_sum).values  
        df_features['{}_weekday_absmean_{}_{}'.format(key, i + 1, 'visitorsdiff')]   = df_features[key].map(dic_abs_mean).values  

    return df_features

#### air_store_id + visitors exp特征(周期特征)
加权指数衰减统计特征

权重可以表示为店铺的运营的时间信息，所以记录一次即可

In [28]:
def get_store_id_visitors_exp_features(df, en_date, key='air_store_id', date_col='visit_date', days_list=[1000]): #, 56, 28, 14,
    """
    时间窗口，groupby对象不同，这里用的是key=air_store_id_weekday
    1. 时间窗口
    2. day_lags = visit_date与en_date相差的天数
    3. day_lags的权重 = weight ** day_lags
    4. visitors_weight = visitors * weight
    5. 对weight(加权日期差之和)求和
    6. 加权平均每天的visitors
    """
    t0 = time()
    df_features       = pd.DataFrame()
    df_features[key]  = df[key].unique()

    df_tmp            = df.copy()
    for weight in [0.9, 0.95, 0.975, 0.985]:
        for days in days_list:
            st_date            = get_st_date(en_date, days) 
            df_tmp             = df_tmp.loc[((df[date_col] >= st_date) & (df[date_col] < en_date))].copy()

            # en_date与visit_date两个日期相差的天数
            df_tmp['day_lags']        = df_tmp['visit_date'].apply(lambda x: (en_date - (x)).days)
            # 相差的天数做一个权重，weight ** day_lags
            df_tmp['weight']          = df_tmp['day_lags'].apply(lambda x: weight ** x)
            # visitors * weight
            df_tmp['visitors_weight'] = df_tmp['visitors'] * df_tmp['weight']
            
            # groupby(air_store_id), 对visitors_weight(加权visitors之和)求和
            result1 = df_tmp.groupby([key], as_index=False)['visitors_weight'].agg({
                '{}_exp_mean{}_w_{}'.format(key, days, weight): 'sum'
            })
            
            # groupby(air_store_id), 对weight(加权日期差之和)求和
            result2 = df_tmp.groupby([key], as_index=False)['weight'].agg({
                '{}_exp_weight_sum{}_w_{}'.format(key, days, weight): 'sum'
            })

            result = result1.merge(result2, on=key, how='left')
            # 加权平均每天的visitors
            result['{}_exp_mean2{}_w_{}'.format(key, days, weight)] = (result['{}_exp_mean{}_w_{}'.format(key, days, weight)]
                                                                       / result['{}_exp_weight_sum{}_w_{}'.format(key, days, weight)])

            if weight == 0.9:
                df_features = df_features.merge(result[[
                    '{}_exp_mean2{}_w_{}'.format(key, days, weight),  # 加权平均每天的visitors
                    '{}_exp_weight_sum{}_w_{}'.format(key, days, weight),  # 对weight(加权日期差之和)求和，只需记录一次
                    key  # air_store_id
                ]], on=key, how='left')
            else:
                df_features = df_features.merge(result[[
                    '{}_exp_mean2{}_w_{}'.format(key, days, weight),  # 加权平均每天的visitors
                    key  # air_store_id
                ]], on=key, how='left')
    print('For store id exp features, we spend {} seconds.'.format(time() - t0)) 

    return df_features

#### air_store_id + weekday + visitors exp特征(7)(周期特征)
加权指数衰减(按周进行衰减)统计特征

权重可以表示为店铺的运营的时间信息(周)，所以记录一次即可

In [29]:
def get_weekday_visitors_exp_sts_features(df, en_date, key, date_col='visit_date', days_list=[1000]): #, 56, 28, 14,
    """
    时间窗口
    1. 时间窗口
    2. day_lags = (visit_date与en_date相差的天数) // 7
    3. weight = weight ** day_lags
    4. visitors_weight = visitors * weight
    5. visitors_weight求和
    6. weight求和
    7. 加权平均每天的visitors
    """
    t0 = time()
    df_features       = pd.DataFrame()
    df_features[key]  = df[key].unique()
    
    df_tmp            = df.copy()
    for weight in [0.95, 0.975, 0.985]:   
        for days in days_list:
            # 时间窗口
            st_date            = get_st_date(en_date, days) 
            df_tmp             = df_tmp.loc[((df[date_col] >= st_date) & (df[date_col] < en_date))].copy()

            # day_lags = (en_date - visit_date) //7
            df_tmp['day_lags']        = df_tmp['visit_date'].apply(lambda x: (en_date - (x)).days // 7)
            # day_lags加权
            df_tmp['weight']          = df_tmp['day_lags'].apply(lambda x: weight ** x)
            # visitors_weight = visitors * weight
            df_tmp['visitors_weight'] = df_tmp['visitors'] * df_tmp['weight']
            
            # visitors_weight求和
            result1 = df_tmp.groupby([key], as_index=False)['visitors_weight'].agg({
                '{}_exp_mean{}_{}'.format(key, days, weight): 'sum'
            })
            # weight求和
            result2 = df_tmp.groupby([key], as_index=False)['weight'].agg({
                '{}_exp_weight_sum{}_{}'.format(key,days,weight): 'sum'
            })

            result = result1.merge(result2, on=key, how='left')
            
            # 加权平均每天的visitors
            result['{}_exp_mean2{}_{}'.format(key,days,weight)] = (result['{}_exp_mean{}_{}'.format(key,days,weight)]
                                                                   / result['{}_exp_weight_sum{}_{}'.format(key,days,weight)])
            
            df_features = df_features.merge(result, on=key, how='left')
    print('For store id exp features, we spend {} seconds.'.format(time() - t0)) 
    
    return df_features

#### air_store_id + holiday +visitors特征
过去每个节假日的的统计特征

In [30]:
def get_holiday_visitors_sts_features(df, key, end_date, date_col='visit_date', val_name='visitors'): 
    """
    时间窗口，groupby(air_store_id_holiday)
    时间窗口内的统计量
    """
    df_features      = pd.DataFrame() 
    df_features[key] = df[key].unique() 
    ind1    = df[date_col] < end_date  
    for i in [4, 8, 12, 100]: 
        # 时间窗口，前4、8、12、100周
        t_st_date           = end_date - timedelta(7 * i) 
        ind2                = df[date_col]  >= t_st_date 
        df_grp              = df.loc[(ind1 & ind2)].groupby(key) 
        
        # 统计量
        dic_mean            = df_grp[val_name].mean().to_dict()
        dic_std             = df_grp[val_name].std().to_dict()
        dic_max             = df_grp[val_name].max().to_dict()
        dic_min             = df_grp[val_name].min().to_dict()
        dic_count           = df_grp[val_name].count().to_dict()
        dic_sum             = df_grp[val_name].sum().to_dict()   
        df_features['{}_holiday_mean_{}_{}'.format(key, i + 1, val_name)]  = df_features[key].map(dic_mean).values 
        df_features['{}_holiday_std_{}_{}'.format(key, i + 1, val_name)]   = df_features[key].map(dic_std).values 
        df_features['{}_holiday_max_{}_{}'.format(key, i + 1, val_name)]   = df_features[key].map(dic_max).values 
        df_features['{}_holiday_min_{}_{}'.format(key, i + 1, val_name)]   = df_features[key].map(dic_min).values 
        df_features['{}_holiday_count_{}_{}'.format(key, i + 1, val_name)] = df_features[key].map(dic_count).values 
        df_features['{}_holiday_sum_{}_{}'.format(key, i + 1, val_name)]   = df_features[key].map(dic_sum).values  
    return df_features

#### air_store_id + weekday + holiday +visitors特征
过去每个节假日与工作日的交叉统计特征

In [31]:
def get_weekday_holiday_visitors_sts_features(df, key, end_date, date_col='visit_date', val_name='visitors'): 
    """
    时间窗口，groupby(air_store_id_weekday_holiday)
    窗口内的统计量
    """
    df_features      = pd.DataFrame() 
    df_features[key] = df[key].unique() 
    ind1    = df[date_col] < end_date  
    for i in [4, 8, 12, 100]: 
        t_st_date           = end_date - timedelta(7 * i) 
        ind2                = df[date_col] >= t_st_date 
        df_grp              = df.loc[(ind1 & ind2)].groupby(key) 
        dic_mean            = df_grp[val_name].mean().to_dict()
        dic_std             = df_grp[val_name].std().to_dict()
        dic_max             = df_grp[val_name].max().to_dict()
        dic_min             = df_grp[val_name].min().to_dict()
        dic_quantile        = df_grp[val_name].quantile().to_dict()
        # dic_count           = df_grp[val_name].count().to_dict()
        dic_sum             = df_grp[val_name].sum().to_dict()   
        df_features['{}_weekday_holiday_mean_{}_{}'.format(key, i + 1, val_name)]       = df_features[key].map(dic_mean).values 
        df_features['{}_weekday_holiday_std_{}_{}'.format(key, i + 1, val_name)]        = df_features[key].map(dic_std).values 
        df_features['{}_weekday_holiday_max_{}_{}'.format(key, i + 1, val_name)]        = df_features[key].map(dic_max).values 
        df_features['{}_weekday_holiday_min_{}_{}'.format(key, i + 1, val_name)]        = df_features[key].map(dic_min).values 
        # df_features['{}_weekday_holiday_count_{}_{}'.format(key, i + 1, val_name)]    = df_features[key].map(dic_count).values 
        df_features['{}_weekday_holiday_sum_{}_{}'.format(key, i + 1, val_name)]        = df_features[key].map(dic_sum).values  
        df_features['{}_weekday_holiday_quantile_{}_{}'.format(key, i + 1, val_name)]   = df_features[key].map(dic_quantile).values  
    return df_features

#### air_store_id + timediff
过去每个商店关于时间的差值统计特征

In [32]:
def get_store_id_time_diff_sts_features(df, en_date, key='air_store_id', date_col='visit_date', days_list=[1000, 56, 28]): #, 14,
    """
    时间窗口，时间差
    1. 时间窗口
    2. 窗口内visit_date的diff
    3. diff的统计量
    """
    t0 = time()
    df_features       = pd.DataFrame()
    df_features[key]  = df[key].unique()
    df = df.sort_values([key, date_col])
    for days in days_list:    
        df_tmp                = df.copy()       
        # 时间窗口
        st_date               = get_st_date(en_date, days) 
        df_tmp                = df_tmp.loc[((df[date_col] >= st_date) & (df[date_col] < en_date))].copy()
        
        # 根据值排序
        df_tmp                = df_tmp.sort_values([key, date_col])
        # 当前项减前一项
        df_tmp['time_diff']   = df_tmp.groupby(key)[date_col].diff().values
        df_tmp['time_diff']   = df_tmp['time_diff'].apply(lambda x: x.days) 
        
        df_tmp                = df_tmp.loc[df_tmp['time_diff'].notnull()] 
        # 统计量
        df_features_tmp   = df_tmp.groupby([key], as_index=False)['time_diff'].agg({
            '{}_time_diff_min{}'.format(key,days): 'min',
            '{}_time_diff_mean{}'.format(key,days): 'mean',
            '{}_time_diff_median{}'.format(key,days): 'median',
            '{}_time_diff_max{}'.format(key,days): 'max',
            '{}_time_diff_std{}'.format(key,days): 'std',
            '{}_time_diff_quantile{}'.format(key,days): 'quantile',
            '{}_time_diff_skew{}'.format(key,days): 'skew'
        }) 
        df_features  = df_features.merge(df_features_tmp, on=key, how='left')
    print('For store id diff sts features, we spend {} seconds.'.format(time() - t0)) 
    
    return df_features

### 特征工程2（reserve相关特征）
####  air_store_id + visit_date + timediff & reserve_visitors特征
每天的预定的统计信息,包括预定人数以及时间差的统计信息

In [33]:
def get_store_date_reserve_features(df_air_reserves, df_hpg_reserves, en_date, days, key='air_store_id_visit_date'):
    """
    20120-07-11
    """
    t0 = time()
    st_date, en_date = get_label_st_date(en_date, days)
    
    df_air_reserve = df_air_reserves.loc[((df_air_reserves['visit_datetime'] < en_date)
                                          & (df_air_reserves['visit_datetime'] >= st_date)
                                          & (df_air_reserves['reserve_datetime'] < st_date))].copy()
    
    df_hpg_reserve = df_hpg_reserves.loc[((df_hpg_reserves['visit_datetime'] < en_date)
                                          & (df_hpg_reserves['visit_datetime'] >= st_date)
                                          & (df_hpg_reserves['reserve_datetime'] < st_date))].copy()
    
    df_features       = pd.DataFrame()
    df_features[key]  = df_air_reserve[key].unique()  
    
    try:  
        df_air_reserve['time_diff']   = df_air_reserve['visit_datetime'] - df_air_reserve['reserve_datetime']
        df_air_reserve['time_diff']   = df_air_reserve['time_diff'].apply(lambda x: x.days).values
        df_air_timediff_fea = df_air_reserve.groupby(key, as_index = False)['time_diff'].agg({
                                           'store_air_timediff_mean': 'mean',
                                           'store_air_timediff_median': 'median',
                                           'store_air_timediff_max': 'max',
                                           'store_air_timediff_std': 'std'}) 

        df_air_reserve_fea = df_air_reserve.groupby(['air_store_id','visit_date'])['reserve_visitors'].agg({
                                           'store_air_reserve_visitors_sum': 'sum',
                                           'store_air_reserve_visitors_mean': 'mean',
    #                                        'store_air_reserve_visitors_std': 'std',
                                           'store_air_reserve_visitors_max': 'max',
                                           'store_air_reserve_visitors_count': 'count',}) 

        df_air_reserve_fea          = df_air_reserve_fea.unstack().fillna(0).stack().reset_index()    
        df_air_reserve_fea[key]     = df_air_reserve_fea['air_store_id'].astype(str) + '_' + df_air_reserve_fea['visit_date'].astype(str)
        del df_air_reserve_fea['air_store_id']
        del df_air_reserve_fea['visit_date']
        df_features  = df_features.merge(df_air_timediff_fea, on =key, how = 'left')
        df_features  = df_features.merge(df_air_reserve_fea,  on =key, how = 'left')

    except:
        pass
    
    try:
        df_hpg_reserve['time_diff']   = df_hpg_reserve['visit_datetime'] - df_hpg_reserve['reserve_datetime']
        df_hpg_reserve['time_diff']   = df_hpg_reserve['time_diff'].apply(lambda x: x.days).values
        
        df_hpg_timediff_fea = df_hpg_reserve.groupby(key, as_index = False)['time_diff'].agg({
                                           'store_hpg_timediff_mean': 'mean',
                                           'store_hpg_timediff_median': 'median',
                                           'store_hpg_timediff_max': 'max',
                                           'store_hpg_timediff_std': 'std',}) 

        df_hpg_reserve_fea = df_hpg_reserve.groupby(['air_store_id','visit_date'])['reserve_visitors'].agg({
                                           'store_hpg_reserve_visitors_sum': 'sum',
                                           'store_hpg_reserve_visitors_mean': 'mean',
    #                                        'store_hpg_reserve_visitors_std': 'std',
                                           'store_hpg_reserve_visitors_max': 'max',
                                           'store_hpg_reserve_visitors_count': 'count',}) 

        df_hpg_reserve_fea = df_hpg_reserve_fea.unstack().fillna(0).stack().reset_index()    
        df_hpg_reserve_fea[key]     = df_hpg_reserve_fea['air_store_id'].astype(str) + '_' + df_hpg_reserve_fea['visit_date'].astype(str)
        del df_hpg_reserve_fea['air_store_id']
        del df_hpg_reserve_fea['visit_date']
        df_features  = df_features.merge(df_hpg_timediff_fea, on =key, how = 'left')
        df_features  = df_features.merge(df_hpg_reserve_fea,  on =key, how = 'left')
        
    except:
        pass
    
    print('For store id reserve visitors features, we spend {} seconds.'.format(time() - t0)) 
    
    return df_features

#### air_store_id + timediff & reserve_visitors特征
整体的预定统计信息,包括预定人数以及时间差的统计信息

In [34]:
def get_store_reserve_features(df_air_reserves, df_hpg_reserves, en_date, days, key = 'air_store_id'):
     
    t0 = time()
    
    st_date,en_date = get_label_st_date(en_date, days) 
   
    df_air_reserve  = df_air_reserves.loc[((df_air_reserves['visit_datetime']   < en_date)  &\
                                           (df_air_reserves['visit_datetime']   >= st_date) &\
                                           (df_air_reserves['reserve_datetime'] < st_date))].copy()
    df_hpg_reserve  = df_hpg_reserves.loc[((df_hpg_reserves['visit_datetime']   < en_date)  &\
                                           (df_hpg_reserves['visit_datetime']   >= st_date) &\
                                           (df_hpg_reserves['reserve_datetime'] < st_date))].copy()
    
    
    df_features       = pd.DataFrame()
    df_features[key]  = df_air_reserve[key].unique()
    
    df_air_reserve['time_diff']   = df_air_reserve['visit_datetime'] - df_air_reserve['reserve_datetime']
    df_air_reserve['time_diff']   = df_air_reserve['time_diff'].apply(lambda x: x.days)
    
    df_hpg_reserve['time_diff']   = df_hpg_reserve['visit_datetime'] - df_hpg_reserve['reserve_datetime']
    df_hpg_reserve['time_diff']   = df_hpg_reserve['time_diff'].apply(lambda x: x.days)
    
    try:
        df_air_timediff_fea = df_air_reserve.groupby(key, as_index = False)['time_diff'].agg({
                                           'store2_air_timediff_mean': 'mean',
                                           'store2_air_timediff_median': 'median',
                                           'store2_air_timediff_max': 'max',
                                           'store2_air_timediff_count': 'count',
                                           'store2_air_timediff_std': 'std',}) 

        df_air_reserve_fea = df_air_reserve.groupby(key, as_index = False)['reserve_visitors'].agg({
                                           'store2_air_reserve_visitors_sum': 'sum',
                                           'store2_air_reserve_visitors_median': 'median',
                                           'store2_air_reserve_visitors_mean': 'mean',
                                           'store2_air_reserve_visitors_max': 'max',
                                           'store2_air_reserve_visitors_count': 'count',
                                           'store2_air_reserve_visitors_std': 'std',}) 
        df_features  = df_features.merge(df_air_timediff_fea, on =key, how = 'left')
        df_features  = df_features.merge(df_air_reserve_fea,  on =key, how = 'left')
        
    except:
        pass
    try:
        df_hpg_timediff_fea = df_hpg_reserve.groupby(key, as_index = False)['time_diff'].agg({
                                           'store2_hpg_timediff_mean': 'mean',
                                           'store2_hpg_timediff_median': 'median',
                                           'store2_hpg_timediff_max': 'max',
                                           'store2_hpg_timediff_count': 'count',
                                           'store2_hpg_timediff_skew': 'skew',
                                           'store2_hpg_timediff_std': 'std',}) 

        df_hpg_reserve_fea = df_hpg_reserve.groupby(key, as_index = False)['reserve_visitors'].agg({
                                           'store2_hpg_reserve_visitors_sum': 'sum',
                                           'store2_hpg_reserve_visitors_mean': 'mean',
                                           'store2_hpg_reserve_visitors_median': 'median',
                                           'store2_hpg_reserve_visitors_max': 'max',
                                           'store2_hpg_reserve_visitors_count': 'count',
                                           'store2_hpg_reserve_visitors_std': 'std',}) 
    
        df_features  = df_features.merge(df_hpg_timediff_fea, on =key, how = 'left')
        df_features  = df_features.merge(df_hpg_reserve_fea,  on =key, how = 'left')
    except:
        pass
    
    print('For store id reserve visitors features, we spend {} seconds.'.format(time() - t0)) 
    
    return df_features

#### visit_date + timediff & reserve_visitors特征
每个时间段的统计信息,包括预定人数以及时间差的统计信息

In [35]:
def get_date_reserve_features(df_air_reserves, df_hpg_reserves, en_date, days, key = 'visit_date'):
     
    t0 = time()
    st_date,en_date = get_label_st_date(en_date, days) 

    df_air_reserve  = df_air_reserves.loc[((df_air_reserves['visit_datetime'] < en_date)  &\
                                           (df_air_reserves['visit_datetime'] >= st_date) &\
                                           (df_air_reserves['reserve_datetime'] < st_date))].copy()
    
    df_hpg_reserve  = df_hpg_reserves.loc[((df_hpg_reserves['visit_datetime'] < en_date)  &\
                                           (df_hpg_reserves['visit_datetime'] >= st_date) &\
                                           (df_hpg_reserves['reserve_datetime'] < st_date))].copy()
    
    
    df_features       = pd.DataFrame()
    df_features[key]  = df_air_reserve[key].unique()
    
    df_air_reserve['time_diff']   = df_air_reserve['visit_datetime'] - df_air_reserve['reserve_datetime']
    df_air_reserve['time_diff']   = df_air_reserve['time_diff'].apply(lambda x: x.days)
    
    df_hpg_reserve['time_diff']   = df_hpg_reserve['visit_datetime'] - df_hpg_reserve['reserve_datetime']
    df_hpg_reserve['time_diff']   = df_hpg_reserve['time_diff'].apply(lambda x: x.days)
    try:
        df_air_timediff_fea = df_air_reserve.groupby(key, as_index = False)['time_diff'].agg({
            'date_air_timediff_mean': 'mean',
            'date_air_timediff_median': 'median',
            'date_air_timediff_max': 'max',
            'date_air_timediff_count': 'count',
            'date_air_timediff_std': 'std',
        }) 

        df_air_reserve_fea = df_air_reserve.groupby(key, as_index = False)['reserve_visitors'].agg({
            'date_air_reserve_visitors_sum': 'sum',
            'date_air_reserve_visitors_median': 'median',
            'date_air_reserve_visitors_max': 'max',
            'date_air_reserve_visitors_count': 'count',
            'date_air_reserve_visitors_std': 'std',
        }) 
        df_features  = df_features.merge(df_air_timediff_fea, on =key, how = 'left')
        df_features  = df_features.merge(df_air_reserve_fea,  on =key, how = 'left')
        
    except:
        pass
    try:
        df_hpg_timediff_fea = df_hpg_reserve.groupby(key, as_index = False)['time_diff'].agg({
            'date_hpg_timediff_mean': 'mean',
            'date_hpg_timediff_max': 'max',
        }) 

        df_hpg_reserve_fea = df_hpg_reserve.groupby(key, as_index = False)['reserve_visitors'].agg({
            'date_hpg_reserve_visitors_sum': 'sum',
            'date_hpg_reserve_visitors_median': 'median',
            'date_hpg_reserve_visitors_max': 'max',
            'date_hpg_reserve_visitors_count': 'count',
        }) 
    
        df_features  = df_features.merge(df_hpg_timediff_fea, on =key, how = 'left')
        df_features  = df_features.merge(df_hpg_reserve_fea,  on =key, how = 'left')
    except:
        pass
    
    print('For store id reserve visitors features, we spend {} seconds.'.format(time() - t0)) 
    
    return df_features

### 特征工程3：基于其他属性的特征,例如air_genre_name
#### air_genre_name + visitors特征
最近N(N=14,28,56,1000)天air_genre_name的统计特征

In [36]:
def get_genre_visitors_features(df, en_date, key = 'air_genre_name', date_col = 'visit_date',  days_list = [1000, 56, 28]): #, 14,
    t0 = time()
    df_features       = pd.DataFrame()
    df_features[key]  = df[key].unique()
    
    df_tmp            = df.copy()
    for days in days_list:
        st_date           = get_st_date(en_date, days) 
        df_tmp            = df_tmp.loc[((df[date_col] >= st_date) &  (df[date_col] < en_date))].copy()
        df_features_tmp   = df_tmp.groupby([key], as_index=False)['visitors'].agg({
            '{}_visitors_min{}'.format(key,days): 'min',
            '{}_visitors_mean{}'.format(key,days): 'mean',
            '{}_visitors_median{}'.format(key,days): 'median',
            '{}_visitors_max{}'.format(key,days): 'max',
            '{}_visitors_sum{}'.format(key,days): 'sum',
            '{}_visitors_std{}'.format(key,days): 'std',
            '{}_visitors_quantile{}'.format(key,days): 'quantile',
            '{}_visitors_skew{}'.format(key,days): 'skew'
        }) 
        df_features  = df_features.merge(df_features_tmp, on =key, how = 'left')
    print('For area sts features, we spend {} seconds.'.format(time() - t0)) 
    
    return df_features

#### air_genre_name + weekday + visitors特征
过去每段时间的工作日的统计特征天air_genre_name对应的统计特征

In [37]:
def get_genre_weekday_visitors_sts_features(df, end_date, key = 'air_genre_name_weekday', date_col = 'visit_date',val_name = 'visitors'): 
    df_features      = pd.DataFrame() 
    df_features[key] = df[key].unique()
    t0 = time()
    ind1    = df[date_col] < end_date  
    for i in [4,8,12,100]:  #2,6
        t_st_date           = end_date  - timedelta(7 * i) 
        ind2                = df[date_col]  >= t_st_date 
        df_grp              = df.loc[(ind1 & ind2)].groupby(key) 
        dic_mean            = df_grp[val_name].mean().to_dict()
        dic_std             = df_grp[val_name].std().to_dict()
        dic_max             = df_grp[val_name].max().to_dict()
        dic_min             = df_grp[val_name].min().to_dict()
        dic_count           = df_grp[val_name].count().to_dict()
        dic_sum             = df_grp[val_name].sum().to_dict()   
        dic_quantile        = df_grp[val_name].quantile().to_dict()   
        
        df_features['{}_mean_{}_{}'.format(key, i+1, val_name)]  = df_features[key].map(dic_mean).values 
        df_features['{}_quantile_{}_{}'.format(key, i+1, val_name)]  = df_features[key].map(dic_quantile).values 
        df_features['{}_std_{}_{}'.format(key, i+1, val_name)]   = df_features[key].map(dic_std).values 
        df_features['{}_max_{}_{}'.format(key, i+1, val_name)]   = df_features[key].map(dic_max).values 
        df_features['{}_min_{}_{}'.format(key, i+1, val_name)]   = df_features[key].map(dic_min).values 
        df_features['{}_gap_{}_{}'.format(key, i+1, val_name)]   = df_features['{}_max_{}_{}'.format(key, i+1, val_name)] - df_features['{}_min_{}_{}'.format(key, i+1, val_name)]
        df_features['{}_count_{}_{}'.format(key, i+1, val_name)] = df_features[key].map(dic_count).values 
        df_features['{}_sum_{}_{}'.format(key, i+1, val_name)]   = df_features[key].map(dic_sum).values  
    print('For area name sts features, we spend {} seconds.'.format(time() - t0)) 
    return df_features

#### air_area_name + weekday + holiday + visitors特征
最近N(N=14,28,56,1000)天air_area_name+weekday以及holiday的统计特征

In [38]:
def get_genre_weekday_holiday_visitors_sts_features(df, end_date, key = 'air_genre_name_weekday_holiday', date_col = 'visit_date',val_name = 'visitors'): 
    df_features      = pd.DataFrame() 
    df_features[key] = df[key].unique()
    t0 = time()
    ind1    = df[date_col] < end_date  
    for i in [4,8,12,100]:  #2,6
        t_st_date           = end_date  - timedelta(7 * i) 
        ind2                = df[date_col]  >= t_st_date 
        df_grp              = df.loc[(ind1 & ind2)].groupby(key) 
        dic_mean            = df_grp[val_name].mean().to_dict()
        dic_std             = df_grp[val_name].std().to_dict()
        dic_max             = df_grp[val_name].max().to_dict()
        dic_min             = df_grp[val_name].min().to_dict()
        dic_count           = df_grp[val_name].count().to_dict()
        dic_sum             = df_grp[val_name].sum().to_dict()   
        dic_quantile        = df_grp[val_name].quantile().to_dict()   
        
        
        df_features['{}_mean_{}_{}'.format(key, i+1, val_name)]  = df_features[key].map(dic_mean).values 
        df_features['{}_quantile_{}_{}'.format(key, i+1, val_name)]  = df_features[key].map(dic_quantile).values 
        df_features['{}_std_{}_{}'.format(key, i+1, val_name)]   = df_features[key].map(dic_std).values 
        df_features['{}_max_{}_{}'.format(key, i+1, val_name)]   = df_features[key].map(dic_max).values 
        df_features['{}_min_{}_{}'.format(key, i+1, val_name)]   = df_features[key].map(dic_min).values 
        df_features['{}_gap_{}_{}'.format(key, i+1, val_name)]   = df_features['{}_max_{}_{}'.format(key, i+1, val_name)] - df_features['{}_min_{}_{}'.format(key, i+1, val_name)]
        df_features['{}_count_{}_{}'.format(key, i+1, val_name)] = df_features[key].map(dic_count).values 
        df_features['{}_sum_{}_{}'.format(key, i+1, val_name)]   = df_features[key].map(dic_sum).values  
    print('For area name sts features, we spend {} seconds.'.format(time() - t0)) 
    return df_features

## 提取特征

In [39]:
def get_features(df_visitors, air_reserve, hpg_air_reserve, label_st_date, days =39, is_te = False):
     
    df_label = get_df_label(df_visitors, label_st_date, days, is_te, ) 
    df_label['visit_date']                         = pd.to_datetime(df_label['visit_date'])
    df_label['weekday']                            = df_label['visit_date'].dt.weekday  
    df_label['air_store_id_weekday']               = df_label['air_store_id'].astype(str) + '_' + df_label['weekday'].astype(str) 
    df_label['air_store_id_weekday_holiday']       = df_label['air_store_id_holiday'] + '_' + df_label['weekday'].astype(str) 
    
    df_label['air_store_id_visit_date']  = df_label['air_store_id']  + '_' + df_label['visit_date'].astype(str)
    ####################
    df_genre_visitors_fea             = get_genre_visitors_features(df=df_visitors, key='air_genre_name', en_date = label_st_date, date_col = 'visit_date')
    df_genre_dow_visitors_fea         = get_genre_weekday_visitors_sts_features(df=df_visitors, key='air_genre_name_weekday', end_date = label_st_date, date_col = 'visit_date',val_name = 'visitors')
    df_genre_dow_holiday_visitors_fea = get_genre_weekday_holiday_visitors_sts_features(df=df_visitors, key='air_genre_name_weekday_holiday', end_date = label_st_date, date_col = 'visit_date',val_name = 'visitors')
     #################### 
    df_air_hpg_features  = get_store_date_reserve_features(df_air_reserves = air_reserve,df_hpg_reserves = hpg_air_reserve, en_date=label_st_date, days=days,) 
    df_air_hpg_features2 = get_store_reserve_features(df_air_reserves = air_reserve,df_hpg_reserves = hpg_air_reserve, en_date=label_st_date, days=days,) 
    df_air_hpg_features3 = get_date_reserve_features(df_air_reserves = air_reserve,df_hpg_reserves = hpg_air_reserve, en_date=label_st_date, days=days,) 
    
    df_storeid_visitors_fea           = get_store_id_visitors_sts_features(df=df_visitors,key='air_store_id', en_date=label_st_date)
    
    df_storeid_timediff_fea           = get_store_id_time_diff_sts_features(df=df_visitors, key='air_store_id',en_date=label_st_date)
     
    df_storeid_visitorsdiff_fea       = get_store_id_visitors_diff_sts_features(df=df_visitors, key='air_store_id', en_date=label_st_date)
    df_storeid_visitorsexp_fea        = get_store_id_visitors_exp_features(df=df_visitors, key='air_store_id', en_date=label_st_date)
    
    df_storeid_dow_visitors_fea       = get_weekday_visitors_sts_features(df=df_visitors,      key='air_store_id_weekday', end_date = label_st_date, date_col = 'visit_date',val_name = 'visitors')
    df_storeid_dow_visitorsdiff_fea   = get_weekday_visitors_diff_sts_features(df=df_visitors, key='air_store_id_weekday', end_date = label_st_date, date_col = 'visit_date',val_name = 'visitors')
    df_storeid_dow_visitorsexp_fea    = get_weekday_visitors_exp_sts_features(df=df_visitors,  key='air_store_id_weekday', en_date = label_st_date, date_col = 'visit_date')
     
    df_storeid_holiday_visitors_fea   = get_holiday_visitors_sts_features(df=df_visitors, key='air_store_id_holiday', end_date = label_st_date, date_col = 'visit_date',val_name = 'visitors')
    df_storeid_dow_holiday_visitorsdiff_fea   = get_weekday_holiday_visitors_sts_features(df=df_visitors, key='air_store_id_weekday_holiday', end_date = label_st_date, date_col = 'visit_date',val_name = 'visitors')
     
    df_features             = df_label.merge(df_storeid_visitors_fea,            on =['air_store_id'], how='left')
    df_features             = df_features.merge(df_storeid_timediff_fea,         on =['air_store_id'], how='left')   
    df_features             = df_features.merge(df_storeid_visitorsdiff_fea,     on =['air_store_id'], how='left')  
    df_features             = df_features.merge(df_storeid_visitorsexp_fea,      on =['air_store_id'], how='left')  
    
    df_features             = df_features.merge(df_storeid_dow_visitors_fea,     on =['air_store_id_weekday'], how='left')  
    df_features             = df_features.merge(df_storeid_dow_visitorsdiff_fea, on =['air_store_id_weekday'], how='left')  
    df_features             = df_features.merge(df_storeid_dow_visitorsexp_fea,  on =['air_store_id_weekday'], how='left')   
     
    
    df_features             = df_features.merge(df_air_hpg_features,  on =['air_store_id_visit_date'], how='left')
    df_features             = df_features.merge(df_air_hpg_features2, on =['air_store_id'], how='left') 
    df_features['visit_date']          = df_features['visit_date'].astype(str)
    df_air_hpg_features3['visit_date'] = df_air_hpg_features3['visit_date'].astype(str)  
    df_features             = df_features.merge(df_air_hpg_features3, on =['visit_date'], how='left') 
    
    df_features             = df_features.merge(df_storeid_holiday_visitors_fea, on =['air_store_id_holiday'], how='left')  
    df_features             = df_features.merge(df_storeid_dow_holiday_visitorsdiff_fea, on =['air_store_id_weekday_holiday'], how='left')  
    
    df_features             = df_features.merge(df_genre_visitors_fea, on =['air_genre_name'], how='left')   
    df_features             = df_features.merge(df_genre_dow_visitors_fea, on =['air_genre_name_weekday'], how='left')  
    df_features             = df_features.merge(df_genre_dow_holiday_visitors_fea, on =['air_genre_name_weekday_holiday'], how='left')
    
    del df_features['air_genre_name'] 
    return df_features

## 模型验证
### 模型验证的框架

In [40]:
def lgb_model_test_weight_timebased(df_tr, df_val, features, ws = [1.01], test_df = None): 
    params = {'num_leaves': 256,
          'min_child_samples': 79,
          'objective': 'rmse',
          'max_depth': 13,
          'learning_rate': 0.03,
          "boosting_type": "gbdt",
          "subsample_freq": 3,
          "subsample": 0.9,
          "bagging_seed": 11,
          "metric": 'rmse',
          "verbosity": -1,
          'reg_alpha': 0.3,
          'reg_lambda': 0.3,
          'colsample_bytree': 0.9, 
         } 

    MAX_ROUNDS = 700
    val_pred = []
    test_pred = []
    cate_vars = []
    feature_importance = None
    models = []  
     
    train_df,train_label = df_tr[features].copy() ,df_tr['visitors'].values
    val_df,val_label     = df_val[features].copy(),df_val['visitors'].values
    
    for i,w in enumerate(ws):
    
        train_weight = []
        tmp = df_tr['day_to_now'].values
        for d in tmp:
            if d > 365:
                train_weight.append(1) 
            else:
                train_weight.append(w ** ((365 - d) // (60) )) 
        
        dtrain = lgb.Dataset(train_df, label = train_label, weight=train_weight)
        dval   = lgb.Dataset(val_df,   label = val_label, reference=dtrain)  
        
        bst = lgb.train(params, dtrain, num_boost_round=MAX_ROUNDS,valid_sets=[dtrain,dval],  early_stopping_rounds=1500, verbose_eval=100)
        models.append(bst)
        f_importance = pd.DataFrame()
        f_importance['fea'] = features
        f_importance['imp'] = bst.feature_importance("gain") 
        f_importance['fold'] = i
        if feature_importance is None:
            feature_importance = f_importance
        else:
            feature_importance = pd.concat([feature_importance, f_importance],axis=0,ignore_index=True)
  
        if test_df is not None:
            test_pred.append(bst.predict(
                test_df[features], num_iteration=bst.best_iteration or MAX_ROUNDS))
    if test_df is None:
        return models, feature_importance
    else:
        return models, feature_importance, test_pred

### 模型验证
#### 加入store date相关的reserve特征

In [41]:
df_tr_feas = pd.DataFrame()
tr_fea_list = []
for i in tqdm(range(58)): 
    try:
        tr_label_date      = parse('2017-04-23') - timedelta(7*6 + 7 *i)   #*2
        tr_fea_date        = parse('2017-04-23') - timedelta(7*6 + 7 *i+1) #**2

        df_tr_tmp          = get_features(df_meta, air_reserve, hpg_air_reserve, label_st_date= tr_label_date)    
        df_tr_feas         = pd.concat([df_tr_feas, df_tr_tmp], axis=0,ignore_index=True) 
        tr_fea_list.append(df_tr_tmp)
    except:
        print('Problems happen at step {}'.format(i))

  0%|                                                                                           | 0/58 [00:00<?, ?it/s]

label date,  2017-03-12 00:00:00 2017-04-20 00:00:00
For area sts features, we spend 0.27629852294921875 seconds.
For area name sts features, we spend 0.1984727382659912 seconds.
For area name sts features, we spend 0.2064061164855957 seconds.
For store id reserve visitors features, we spend 0.0469202995300293 seconds.
For store id reserve visitors features, we spend 0.07256531715393066 seconds.
For store id reserve visitors features, we spend 0.048873186111450195 seconds.
For store id sts features, we spend 0.7879033088684082 seconds.
For store id diff sts features, we spend 3.523646354675293 seconds.
For store id diff sts features, we spend 3.1218926906585693 seconds.
For store id exp features, we spend 14.643365621566772 seconds.
For store id exp features, we spend 11.172038555145264 seconds.


  2%|█▍                                                                                 | 1/58 [00:39<37:25, 39.39s/it]

label date,  2017-03-05 00:00:00 2017-04-13 00:00:00
For area sts features, we spend 0.245375394821167 seconds.
For area name sts features, we spend 0.1944746971130371 seconds.
For area name sts features, we spend 0.20363354682922363 seconds.
For store id reserve visitors features, we spend 0.03192901611328125 seconds.
For store id reserve visitors features, we spend 0.06682157516479492 seconds.
For store id reserve visitors features, we spend 0.04487872123718262 seconds.
For store id sts features, we spend 0.7818658351898193 seconds.
For store id diff sts features, we spend 3.426281213760376 seconds.
For store id diff sts features, we spend 3.0157310962677 seconds.
For store id exp features, we spend 14.197028875350952 seconds.
For store id exp features, we spend 10.981896162033081 seconds.


  3%|██▊                                                                                | 2/58 [01:17<36:31, 39.13s/it]

...

 97%|███████████████████████████████████████████████████████████████████████████████▏  | 56/58 [22:28<00:24, 12.09s/it]

label date,  2016-02-14 00:00:00 2016-03-24 00:00:00
For area sts features, we spend 0.10270071029663086 seconds.
For area name sts features, we spend 0.06881570816040039 seconds.
For area name sts features, we spend 0.0759575366973877 seconds.
For store id reserve visitors features, we spend 0.022934913635253906 seconds.
For store id reserve visitors features, we spend 0.05386471748352051 seconds.
For store id reserve visitors features, we spend 0.03685903549194336 seconds.
For store id sts features, we spend 0.3054957389831543 seconds.
For store id diff sts features, we spend 0.7711505889892578 seconds.
For store id diff sts features, we spend 0.5735859870910645 seconds.
For store id exp features, we spend 0.7718179225921631 seconds.
For store id exp features, we spend 0.6054859161376953 seconds.


 98%|████████████████████████████████████████████████████████████████████████████████▌ | 57/58 [22:39<00:11, 11.87s/it]

label date,  2016-02-07 00:00:00 2016-03-17 00:00:00
For area sts features, we spend 0.10369634628295898 seconds.
For area name sts features, we spend 0.06685042381286621 seconds.
For area name sts features, we spend 0.062486886978149414 seconds.
For store id reserve visitors features, we spend 0.03124380111694336 seconds.
For store id reserve visitors features, we spend 0.04686307907104492 seconds.
For store id reserve visitors features, we spend 0.032396793365478516 seconds.
For store id sts features, we spend 0.2933840751647949 seconds.
For store id diff sts features, we spend 0.7631344795227051 seconds.
For store id diff sts features, we spend 0.5026836395263672 seconds.
For store id exp features, we spend 0.6685564517974854 seconds.
For store id exp features, we spend 0.5014147758483887 seconds.


100%|██████████████████████████████████████████████████████████████████████████████████| 58/58 [22:50<00:00, 23.64s/it]


In [42]:
for i in tqdm(range(1,6)): 
    tr_label_date      = parse('2017-04-23') - timedelta(7*6 - 7 *i) 
    df_tr_tmp          = get_features(df_meta, air_reserve, hpg_air_reserve,days=42 - 7*i, label_st_date= tr_label_date)    
    df_tr_feas         = pd.concat([df_tr_feas, df_tr_tmp], axis=0,ignore_index=True) 
    tr_fea_list.append(df_tr_tmp)

val_label_date    = parse('2017-04-23')  - timedelta(7*6) 
te_fea_date       = parse('2017-04-23')
 
df_val = get_features(df_meta,air_reserve, hpg_air_reserve, label_st_date= val_label_date)    
df_te  = get_features(df_meta,air_reserve, hpg_air_reserve, label_st_date= te_fea_date, is_te=True)      

## df_date['visit_date'] = pd.to_datetime(df_date['visit_date'])
df_date['visit_date'] = df_date['visit_date'].astype(str)
df_tr                 = df_tr_feas.merge(df_date, on = ['visit_date'], how = 'left')
df_tr                 = df_tr.merge(df_storeid,   on = ['air_store_id'], how = 'left')  
df_tr['visit_date']   = pd.to_datetime(df_tr['visit_date'])
df_tr['day_to_now']   = df_tr['visit_date'].apply(lambda x : (parse('2017-04-22') - (x)).days) 
 
df_val = df_val.merge(df_date,       on = ['visit_date'], how = 'left')
df_val = df_val.merge(df_storeid,    on = ['air_store_id'], how = 'left')  
 
df_te  = df_te.merge(df_date,       on = ['visit_date'], how = 'left') 
df_te  = df_te.merge(df_storeid,    on = ['air_store_id'], how = 'left')

  0%|                                                                                            | 0/5 [00:00<?, ?it/s]

label date,  2017-03-19 00:00:00 2017-04-23 00:00:00
For area sts features, we spend 0.3222973346710205 seconds.
For area name sts features, we spend 0.21712517738342285 seconds.
For area name sts features, we spend 0.21384453773498535 seconds.
For store id reserve visitors features, we spend 0.04041171073913574 seconds.
For store id reserve visitors features, we spend 0.0781104564666748 seconds.
For store id reserve visitors features, we spend 0.056084632873535156 seconds.
For store id sts features, we spend 0.7814249992370605 seconds.
For store id diff sts features, we spend 3.429210901260376 seconds.
For store id diff sts features, we spend 3.0542123317718506 seconds.
For store id exp features, we spend 14.691834688186646 seconds.
For store id exp features, we spend 11.932658195495605 seconds.


 20%|████████████████▊                                                                   | 1/5 [00:47<03:11, 47.96s/it]

label date,  2017-03-26 00:00:00 2017-04-23 00:00:00
For area sts features, we spend 0.2662825584411621 seconds.
For area name sts features, we spend 0.2343745231628418 seconds.
For area name sts features, we spend 0.2092733383178711 seconds.
For store id reserve visitors features, we spend 0.031969547271728516 seconds.
For store id reserve visitors features, we spend 0.06582236289978027 seconds.
For store id reserve visitors features, we spend 0.04288601875305176 seconds.
For store id sts features, we spend 0.7626745700836182 seconds.
For store id diff sts features, we spend 3.453118324279785 seconds.
For store id diff sts features, we spend 3.137575387954712 seconds.
For store id exp features, we spend 15.348093509674072 seconds.
For store id exp features, we spend 11.249058485031128 seconds.


 40%|█████████████████████████████████▌                                                  | 2/5 [01:34<02:22, 47.64s/it]

...

label date,  2017-04-09 00:00:00 2017-04-23 00:00:00
For area sts features, we spend 0.2622971534729004 seconds.
For area name sts features, we spend 0.2074596881866455 seconds.
For area name sts features, we spend 0.20934510231018066 seconds.
For store id reserve visitors features, we spend 0.012047052383422852 seconds.
For store id reserve visitors features, we spend 0.06254124641418457 seconds.
For store id reserve visitors features, we spend 0.04686379432678223 seconds.
For store id sts features, we spend 0.75888991355896 seconds.
For store id diff sts features, we spend 3.5566561222076416 seconds.
For store id diff sts features, we spend 3.1573891639709473 seconds.
For store id exp features, we spend 15.59132981300354 seconds.
For store id exp features, we spend 11.714913606643677 seconds.


 80%|███████████████████████████████████████████████████████████████████▏                | 4/5 [03:10<00:47, 47.75s/it]

label date,  2017-04-16 00:00:00 2017-04-23 00:00:00
For area sts features, we spend 0.26612162590026855 seconds.
For area name sts features, we spend 0.20562219619750977 seconds.
For area name sts features, we spend 0.2125861644744873 seconds.
For store id reserve visitors features, we spend 0.02498006820678711 seconds.
For store id reserve visitors features, we spend 0.06253457069396973 seconds.
For store id reserve visitors features, we spend 0.031235694885253906 seconds.
For store id sts features, we spend 0.7756028175354004 seconds.
For store id diff sts features, we spend 3.6182496547698975 seconds.
For store id diff sts features, we spend 3.2113964557647705 seconds.
For store id exp features, we spend 15.792046785354614 seconds.
For store id exp features, we spend 12.107356786727905 seconds.


100%|████████████████████████████████████████████████████████████████████████████████████| 5/5 [03:59<00:00, 47.87s/it]


label date,  2017-03-12 00:00:00 2017-04-20 00:00:00
For area sts features, we spend 0.2505829334259033 seconds.
For area name sts features, we spend 0.1954958438873291 seconds.
For area name sts features, we spend 0.20346927642822266 seconds.
For store id reserve visitors features, we spend 0.04492449760437012 seconds.
For store id reserve visitors features, we spend 0.07081985473632812 seconds.
For store id reserve visitors features, we spend 0.0488734245300293 seconds.
For store id sts features, we spend 0.756033182144165 seconds.
For store id diff sts features, we spend 3.4510409832000732 seconds.
For store id diff sts features, we spend 3.134182929992676 seconds.
For store id exp features, we spend 14.393874883651733 seconds.
For store id exp features, we spend 10.774326801300049 seconds.
label date,  2017-04-23 00:00:00 2017-06-01 00:00:00
For area sts features, we spend 0.25017881393432617 seconds.
For area name sts features, we spend 0.22463035583496094 seconds.
For area name s

In [43]:
other_cols   = [col for col in df_tr_feas.columns if df_tr_feas[col].dtypes!='object' and col not in ['air_store_id','visit_date','air_store_id_weekday','air_store_id_holiday','air_store_id_weekday_holiday','visitors','weekday',\
                                                                'air_store_id_visit_date','air_area_name','air_area_name_weekday']]
date_cols    = [col for col in df_date.columns if col not in ['visit_date']]
storeid_cols = [col for col in df_storeid.columns if col not in ['air_store_id']]

le = LabelEncoder()
le.fit(list(set(df_tr['air_store_id'].values)) + list(set(df_val['air_store_id'].values)) + list(set(df_te['air_store_id'].values)))   

df_tr['air_store_id_lb']  = le.transform(df_tr['air_store_id'].values)
df_val['air_store_id_lb'] = le.transform(df_val['air_store_id'].values)
df_te['air_store_id_lb']  = le.transform(df_te['air_store_id'].values)

In [44]:
label_col    = 'visitors' 
feature_cols = date_cols + storeid_cols + other_cols + ['air_store_id_lb']  
feature_cols = [col for col in feature_cols if col not in ['day_to_last_holiday','day_to_next_holiday']]

models1, feature_importance1,test_pred1 = lgb_model_test_weight_timebased(df_tr = df_tr, df_val = df_val,features =feature_cols , ws = [1.0], test_df=df_te)

submit = df_te[['air_store_id','visit_date']].copy()
submit['id'] = submit['air_store_id'] + '_' + submit['visit_date'].astype(str) 
submit['visitors'] =  np.expm1(test_pred1[0])
submit['visitors'] = submit['visitors'] #.astype(int)
submit.loc[submit['visitors'] == 0, 'visitors'] = 1
submit[['id','visitors']].to_csv('../sub/sub.csv',index = None)

Training until validation scores don't improve for 1500 rounds
[100]	training's rmse: 0.512099	valid_1's rmse: 0.499047
[200]	training's rmse: 0.490999	valid_1's rmse: 0.480723
[300]	training's rmse: 0.477733	valid_1's rmse: 0.467757
[400]	training's rmse: 0.467846	valid_1's rmse: 0.457847
[500]	training's rmse: 0.459713	valid_1's rmse: 0.449177
[600]	training's rmse: 0.452696	valid_1's rmse: 0.441954
[700]	training's rmse: 0.446322	valid_1's rmse: 0.435499
Did not meet early stopping. Best iteration is:
[700]	training's rmse: 0.446322	valid_1's rmse: 0.435499
